In [3]:
!pip install yfinance
print("listo")

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 65.3/65.3 kB 320.2 kB/s eta 0:00:00
     ------------------------------------ 112.2/112.2 kB 815.5 kB/s eta 0:00:00
     -------------------------------------- 62.6/62.6 kB 371.2 kB/s eta 0:00:00
listo


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
open-clip-torch 2.20.0 requires ftfy, which is not installed.
open-clip-torch 2.20.0 requires sentencepiece, which is not installed.
open-clip-torch 2.20.0 requires timm, which is not installed.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
open-clip-torch 2.20.0 requires protobuf<4, but you have protobuf 4.23.4 which is incompatible.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.31.0 which is incompatible.


In [1]:
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

print("listo")


listo


In [2]:
# Fetch historical stock data using yfinance
stock_symbol = "AAPL"
start_date = "2020-01-01"
end_date = "2023-01-01"
stock_data = yf.download(stock_symbol, start=start_date, end=end_date, progress=False)

# Extract the 'Close' prices
stock_prices = stock_data["Close"].values.reshape(-1, 1)
print("listo")

listo


## Dividimos los datos 

In [3]:
train_size = int(len(stock_prices) * 0.8)
train_data = stock_prices[:train_size]
test_data = stock_prices[train_size:]
print("listo")

listo


## Normalizamos los datos

In [4]:
scaler = MinMaxScaler()
train_data_normalized = scaler.fit_transform(train_data)
test_data_normalized = scaler.transform(test_data)
print("listo")

listo


## Creamos la secuencia

In [5]:
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]
        sequences.append(seq)
    return np.array(sequences)
print("listo")

listo


## Definimos la longitud de las secuencias de entrada

In [6]:
sequence_length = 10  
train_sequences = create_sequences(train_data_normalized, sequence_length)
test_sequences = create_sequences(test_data_normalized, sequence_length)
print("listo")

listo


# Creamos el modelo LSTM con tres capas LSTM seguida de tres capas densas para regresion

## Definimos el modelo

In [7]:
model = Sequential()
print("listo")

listo


## Agregamos capas LSTM

In [ ]:
model.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, 1)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
print("listo")

listo


## Agregar capas densas de regresión

In [ ]:
model.add(Dense(units=25, activation='relu'))
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=1))  # Capa de salida para la regresión
print("listo")

listo


In [11]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

# Entrenamos el modelo
model.fit(train_sequences, train_data_normalized[sequence_length:], epochs=100, batch_size=64)

print("listo")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 50)            10400     
                                                                 
 lstm_1 (LSTM)               (None, 10, 50)            20200     
                                                                 
 lstm_2 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 25)                1275      
                                                                 
 dense_1 (Dense)             (None, 10)                260       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 52346 (204.48 KB)
Trainable params: 52346 

## Realizamos predicciones y evaluamos el modelo

In [12]:
## Hacer predicciones en el conjunto de prueba
test_predictions = model.predict(test_sequences)

## Invertir la normalización en las predicciones
test_predictions = scaler.inverse_transform(test_predictions)

## Invertir la normalización en los datos de prueba reales
test_actual = scaler.inverse_transform(test_data_normalized[sequence_length:])

## Calcular el MSE y RMSE
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(test_actual, test_predictions)
rmse = np.sqrt(mse)

print(f'MSE: {mse}')
print(f'RMSE: {rmse}')

5/5 [==============================] - 1s 9ms/step
MSE: 52.45588705122748
RMSE: 7.242643650713976
